<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Calibration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calibration calculation
This skript aims to allow the calculation and generation of a linear regression to allow the quantivication of phthalates in the environement.

nput: The the peak list files (classic structure, header = 6, by thermofisher) are uploaded to the directory titeled *calibration_input*, it is of key importance that the title of the files follows the format: Cal_"conc.nr"_"series.nr"_"identification.nr"_pl.csv Any changes to filename, path or other changes that might impact the file significantly need to be comented and marked extensively.

The following code serves to import the librarys and mount the google drive.

In [1]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv

# Mount Google Drive
drive.mount('/content/drive', force_remount = True)

# Set path to the folder containing the CSV files
path = '/content/drive/My Drive/calibration_input'

Mounted at /content/drive


This loop aims at importing the various files from the google drive, looping throug the colums and saving them in a new file.

In [ ]:
#First version seems rather unoptimized, hence version 2 seems a lot better, select that one if it seems to work.
# Get a list of all CSV files in the current directory
files = glob.glob('*.csv')

# Loop over each CSV file
for file in files:
    # Read in the CSV file
    df = pd.read_csv(file, header = 6)
    
    # Loop over each row and save a separate data frame for each row
    for i, row in df.iterrows():
        row_df = pd.DataFrame(row).T
        
        # Save the row data frame as a new CSV file
        row_df.to_csv(f'row{i+1}_{file}', index=False)
        
# Get a list of all CSV files with row data
row_files = glob.glob('row*_file*.csv')

# Create an empty list to store the row data frames
row_dfs = []

# Loop over each row file, read it into a data frame, and append it to the list
for file in row_files:
    row_df = pd.read_csv(file)
    row_dfs.append(row_df)

# Concatenate the row data frames along the rows axis
combined_df = pd.concat(row_dfs, axis=0)

# Save the combined data frame to a CSV file
combined_df.to_csv('combined_rows.csv', index=False)


In [ ]:
#version 2 which seems to be more optimized than version one
# Get a list of all CSV files in the current directory
files = glob.glob('Cal_*_pl.csv')

  # Loop over each row and save a separate data frame for each row
  for i in range(len(pd.read_csv(files[0]).columns)):
      row_dfs = []
      for file in files:
          # Read in the CSV file
          df = pd.read_csv(file)
          
          # Extract the i'th row of the CSV file and append it to the row data frame list
          row = df.iloc[i]
          row_dfs.append(row)
          
      # Concatenate the row data frames along the columns axis and convert to a data frame
      combined_df = pd.concat(row_dfs, axis=1).T
      
      # Save the combined data frame to a CSV file with a name indicating the row number
      combined_df.to_csv('/content/drive/My Drive/calibration_output/row{i+1}.csv', index=False)

main idea

In [17]:
# Get a list of all CSV files in the current directory
files = glob.glob('Cal_*.csv')

# Specify the string prefix and number series in the filename to filter and loop over
string_prefix = 'Cal_'
number_series = [4]
string_suffix = 'ug_ml_*.csv'

# Outer loop to iterate over the number series
for file_filter in number_series:
    filtered_files = [file for file in files if f'{string_prefix}{file_filter}{string_suffix}' in file]

    # Loop over each row and save a separate data frame for each 
    for i in range(len(pd.read_csv(filtered_files[0]).columns)):
        row_dfs = []
        for file in filtered_files:
            # Read in the CSV file
            df = pd.read_csv(file)

            # Extract the i'th row of the CSV file and append it to the row data frame list
            row = df.iloc[i]
            row_dfs.append(row)

        # Concatenate the row data frames along the columns axis and convert to a data frame
        combined_df = pd.concat(row_dfs, axis=1).T

        # Save the combined data frame to a CSV file with a name indicating the row number
        combined_df.to_csv(f'/content/drive/My Drive/calibration_output/{string_prefix}{file_filter}{string_suffix}_row{i+1}.csv', index=False)


Files: []
Filtered Files: []


IndexError: ignored

optimizing/debugging

In [12]:
import glob
import pandas as pd

# Get a list of all CSV files in the current directory
files = glob.glob('/content/drive/My Drive/calibration_input/Cal_*ug_ml_*.csv')

# Check if the files can be opened successfully
opened_files = []
for file in files:
    try:
        df = pd.read_csv(file, header=5)
        opened_files.append(file)
        print(f"Opened file: {file}")
        print(df.head())  # Display the first few rows of the opened file
        print()
    except IOError:
        print(f"Error opening file: {file}")

if len(opened_files) == 0:
    print("No files could be opened. Aborting further processing.")
else:
    # Specify the string prefix and number series in the filename to loop over
    string_prefix = 'Cal_'
    number_list = [4]

    # Loop over the number list
    for number in number_list:
        # Construct the filename pattern based on the string prefix and current number
        file_pattern = f'{string_prefix}{number}ug_ml_*.csv'

        print(f"Filtered Files for {number}:", file_pattern)

        # Filter files based on the string prefix and file pattern
        filtered_files = [file for file in opened_files if file.endswith(file_pattern)]

        if len(filtered_files) == 0:
            print("No files found for the filter.")
            continue

        # Continue with processing the files
        for i in range(len(pd.read_csv(filtered_files[0], header=5))):
            row_dfs = []
            for file in filtered_files:
                # Read in the CSV file
                df = pd.read_csv(file, header=5)

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i]
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/calibration_output/{string_prefix}{number}_row{i+1}.csv', index=False)


Opened file: /content/drive/My Drive/calibration_input/Cal_4ug_ml_3.csv
    4.35   4.31   4.39  65790142350.304  23.52  38623269390.338  25.48
0   6.21   6.17   6.23     2.278409e+10   8.15     1.663377e+10  10.97
1   7.74   7.70   7.77     3.148538e+10  11.26     1.838307e+10  12.13
2  13.33  13.29  13.40     4.362993e+10  15.60     1.804275e+10  11.90
3  18.65  18.60  18.69     3.378076e+10  12.08     1.793001e+10  11.83
4  20.44  20.39  20.54     3.906167e+10  13.97     2.082997e+10  13.74

Opened file: /content/drive/My Drive/calibration_input/Cal_4ug_ml_4.csv
    4.35   4.31   4.39  66717586696.083  23.56  38463262699.681  25.52
0   6.21   6.18   6.24     2.326886e+10   8.22     1.665072e+10  11.05
1   7.74   7.70   7.78     3.203063e+10  11.31     1.862262e+10  12.36
2  13.33  13.28  13.40     4.391215e+10  15.51     1.752313e+10  11.63
3  18.64  18.58  18.72     3.412012e+10  12.05     1.783792e+10  11.84
4  20.43  20.38  20.57     3.977699e+10  14.05     2.067013e+10  13.72

Op

The following lines use the previously created row file and take the average and the standard deviation *(or the error)* and add them to two new files.

In [ ]:
# Get a list of all CSV files in the current directory
files = glob.glob(f'/content/drive/My Drive/calibration_output/cal_*.csv')
#Loop over each dataframe

for file in files:
  #take the mean of the file and save it in a new data frame
  df_mean = pd.mean(file)
  #save this dataframe in a new csv document
  df_mean.to_csv('/content/drive/My Drive/calibration_output/'file'_mean.csv')
  #take the standard deviation of the file and save it in a new data frame
  df_std = pd.std(file)
  #save this dataframe in a new csv document
  df_std.to_csv('/content/drive/My Drive/calibration_output/'file'_std.csv')

Two files contain the internal standard signals are used to take the ratio of the various signals and save the values in a new file. The file needs to be specified so that the internal standard signal correlates to the divisor name, so that this leads to the correct ratio. The first one calculates the ratio of the mean and the bottom one being used for the ratio of the standard deviation, although that might be insuficient and should be substituted by an appropriate error calculation (possibly gaussian errorpropagation).

In [ ]:
# Read divisor value from the divisor file
divisor_file = 'divisor.csv'  # Replace with the name of your divisor file

with open(divisor_file, 'r') as file:
    reader = csv.reader(file)
    divisor = next(reader)[0]  # Assuming the divisor is in the first column

# Get a list of CSV files in the directory
directory = '/content/drive/My Drive/calibration_output'  # Replace with the actual path to the directory containing the CSV files

csv_files = glob.glob(os.path.join(directory, '*_mean.csv'))

# Process each CSV file
for csv_file in csv_files:
    if csv_file == divisor_file:
        continue  # Skip the divisor file itself

    # Read the contents of the CSV file
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)

    # Perform division on the data
    divided_data = [[str(float(value) / float(divisor)) for value in row] for row in data]

    # Create a new file name with the "_ratio" tag
    new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv' #needs to be adapted to fit the final data file name formate

    # Write the divided data to the new CSV file
     with open(new_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(divided_data)

In [ ]:
# Read divisor value from the divisor file
divisor_file = 'divisor.csv'  # Replace with the name of your divisor file

with open(divisor_file, 'r') as file:
    reader = csv.reader(file)
    divisor = next(reader)[0]  # Assuming the divisor is in the first column

# Get a list of CSV files in the directory
directory = '/content/drive/My Drive/calibration_output'  # Replace with the actual path to the directory containing the CSV files

csv_files = glob.glob(os.path.join(directory, '*_std.csv'))

# Process each CSV file
for csv_file in csv_files:
    if csv_file == divisor_file:
        continue  # Skip the divisor file itself

    # Read the contents of the CSV file
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)

    # Perform division on the data
    divided_data = [[str(float(value) / float(divisor)) for value in row] for row in data]

    # Create a new file name with the "_ratio" tag
    new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv' #needs to be adapted to fit the final data file name formate

    # Write the divided data to the new CSV file
     with open(new_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(divided_data)

The following script creates a plot using the previously optained data. A file containing the concentration will be nescessary to associate the y-axis values with the x-axis values.